<a href="https://colab.research.google.com/github/pratikunterwegs/duke-squirrel/blob/master/04_landcover_cortisol_duke.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installs geemap package
import subprocess
import os

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as geemap
except:
    import geemap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

geemap package not installed. Installing ...
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=xCQSDV-h0P2jyHTN2WnhvSMK4wJ-Q6tcbXwBK5nzT28&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjIGA9Nejo4rKqq7lmD8HIA-qQQOiGBEmB_0tNLFOkXScj2NOF64Ak

Successfully saved authorization token.


In [ ]:
# prelim code for earth engine landcover around duke

# get transects from user assets
transects = ee.FeatureCollection("users/pratik_unterwegs/data_cortisol_buffer")
extent = ee.FeatureCollection("users/pratik_unterwegs/data_bboxes_cortisol")

# get buffer
extent_geom = extent.geometry()
extent_geom = extent_geom.buffer(5000)

# define start and end date
start_date = '2020-04-01'
end_date = '2020-10-01'

# filter sentinel by bounding box
# get sentinel data
sentinel = ee.ImageCollection("COPERNICUS/S2")
sentinel_bounds = sentinel.filterBounds(extent_geom)
sentinel_2020 = sentinel_bounds.filterDate(start_date, end_date)

# adding a NDVI band
def add_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('ndvi')
    return image.addBands([ndvi])


# add an NDBI band
def add_ndbi(image):
    ndbi = image.normalizedDifference(['B11', 'B8']).rename('ndbi')
    return image.addBands([ndbi])


# map metrics over sentinel
sentinel_ndvi = sentinel_2020.map(add_ndvi)
sentinel_ndbi = sentinel_2020.map(add_ndbi)

# get median values
ndvi_median = sentinel_ndvi.select('ndvi').median()
ndbi_median = sentinel_ndbi.select('ndbi').median()

# clip to geometry
ndvi_clip = ndvi_median.clip(extent_geom)
ndbi_clip = ndbi_median.clip(extent_geom)

In [ ]:
# Define an sld style color ramp to apply to the image.
sld_ramp = \
  '<RasterSymbolizer>' + \
    '<ColorMap type="ramp" extended="false" >' + \
      '<ColorMapEntry color="#0D0887" quantity="-1" label="0"/>' + \
      '<ColorMapEntry color="#CC4678" quantity="0" label="300" />' + \
      '<ColorMapEntry color="#F0F921" quantity="0.5" label="500" />' + \
    '</ColorMap>' + \
  '</RasterSymbolizer>';

# print layers
vis = {'bands': ['ndvi']}
Map = geemap.Map(center=[36.0005,-78.9], zoom=12)
Map.addLayer(ndbi_clip.sldStyle(sld_ramp), {}, 'builtup')
Map.addLayer(transects)
Map.addLayerControl()

Map

In [ ]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
# for ndvi
out_ndvi_mean = os.path.join(out_dir, 'ndvi_mean_duke_guelph.csv')
out_ndvi_std = os.path.join(out_dir, 'ndvi_std_duke_guelph.csv')

# for ndbi
out_ndbi_mean = os.path.join(out_dir, 'ndbi_mean_duke_guelph.csv')
out_ndbi_std = os.path.join(out_dir, 'ndbi_std_duke_guelph.csv') 

# check for directory and make
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
# for ndvi
geemap.zonal_statistics(ndvi_clip, transects, out_ndvi_mean, statistics_type='MEAN', scale=10)
geemap.zonal_statistics(ndvi_clip, transects, out_ndvi_std, statistics_type='STD', scale=10)

# for ndbi
geemap.zonal_statistics(ndvi_clip, transects, out_ndbi_mean, statistics_type='MEAN', scale=10)
geemap.zonal_statistics(ndvi_clip, transects, out_ndbi_std, statistics_type='STD', scale=10)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /root/Downloads/ndvi_mean_duke_guelph.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /root/Downloads/ndvi_std_duke_guelph.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /root/Downloads/ndbi_mean_duke_guelph.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /root/Downloads/ndbi_std_duke_guelph.csv


In [ ]:
# export image to drive
downConfig = {'scale': 10, "maxPixels": 1.0E13, 'driveFolder': 'duke-squirrels'}  # scale means resolution.
name = "duke_guelph_ndbi_cortisol"
# print(name)
task = ee.batch.Export.image(ndbi_clip, name, downConfig)
task.start()

# export ndvi to drive
name_ndvi = "duke_guelph_ndvi_cortisol"
# print(name)
task_ndvi = ee.batch.Export.image(ndvi_clip, name_ndvi, downConfig)
task_ndvi.start()
